# Battle of the Neighborhoods

## Introduction

This project is for who is planning to open a Coffee House in Ankara, Turkey. This project suggests the best locations for Coffee Houses in Ankara. Ankara is the capital of Turkey with a population of 5M. Turkish culture has deep roots in coffee. Also, it is getting popular because of its affordable prices. Coffee consumption has increased 13% and average consumption is 1.1 kg/person in 2018. 
This report explores which neighborhoods of Ankara have the most as well as the best Coffee Houses. Also, this project answers the questions “Where should I open an Coffee House?” and “Where should I stay If I want a tasty coffee?”

## Data

<ul>
<li>District of Ankara are obtained from https://en.wikipedia.org/wiki/Ankara_Province</li>

<li>Latitude and Longitude values are obtained by using "geocoder".</li>

<li>All data related to locations will be obtaine by using FourSquare API and Python Libraries.</li>
</ul>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
import geocoder

Solving environment: ...working... 
  - anaconda/win-64::conda-4.8.1-py36_0
  - defaults/win-64::conda-4.8.1-py36_0done

# All requested packages already installed.



In [3]:
wiki_link = 'https://en.wikipedia.org/wiki/Ankara_Province'
wiki_page = requests.get(wiki_link, headers = headers)
wiki_page

<Response [200]>

In [4]:
soup = BeautifulSoup(wiki_page.content, 'html.parser')
table = soup.find('table', {'class':'wikitable sortable'}).tbody

In [5]:
rows = table.find_all('tr')

In [7]:
columns = [i.text.replace('\n', '') for i in rows[0].find_all('th')]
columns

['District', 'Population (2017)', 'Area (km²)', 'Density (per km²)']

In [8]:
df_ankara = pd.DataFrame(columns = columns)

In [10]:
for i in range(1, len(rows)):
    tds = rows[i].find_all('td')
    
    if len(tds) == 7:
        values = [tds[0].text, tds[1].text, tds[2].text.replace('\n', ''.replace('\xa0','')), tds[3].text, tds[4].text.replace('\n', ''.replace('\xa0','')), tds[5].text.replace('\n', ''.replace('\xa0','')), tds[6].text.replace('\n', ''.replace('\xa0',''))]
    else:
        values = [td.text.replace('\n', '').replace('\xa0','') for td in tds]
        
        df_ankara = df_ankara.append(pd.Series(values, index = columns), ignore_index = True)

        df_ankara

In [11]:
df_ankara.head()

,District,Population (2017),Area (km²),Density (per km²)
0,Akyurt,32.863,369,89
1,Altındağ,371.366,123,3.019
2,Ayaş,12.289,1.041,12
3,Bala,21.682,1.851,12
4,Beypazarı,48.476,1.697,29


In [12]:
def get_latlng(arcgis_geocoder):
    
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Ankara, Turkey'.format(arcgis_geocoder))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [13]:
districts = df_ankara['District']    
coordinates = [get_latlng(districts) for districts in districts.tolist()]

In [14]:
df_ankara_loc = df_ankara

df_ankara_coordinates = pd.DataFrame(coordinates, columns = ['Latitude', 'Longitude'])
df_ankara_loc['Latitude'] = df_ankara_coordinates['Latitude']
df_ankara_loc['Longitude'] = df_ankara_coordinates['Longitude']

In [15]:
df_ankara_loc.head()

,District,Population (2017),Area (km²),Density (per km²),Latitude,Longitude
0,Akyurt,32.863,369,89,40.13082,33.08719
1,Altındağ,371.366,123,3.019,39.94171,32.85445
2,Ayaş,12.289,1.041,12,40.01516,32.33270
3,Bala,21.682,1.851,12,39.55391,33.12352
4,Beypazarı,48.476,1.697,29,40.16811,31.92052
